In [1]:
# I am very interested to see if the r-value gets better by making a master list, which
# contains the information below, but all teams are included before the r-value is calculated

# looks like that for teams that score more goals, there is a stronger correlation between
# CA or FA and the number of goals they will score. Not sure if this means I should only consider
# the top scoring teams when evaluating the combined r-value or if I should just use this information
# to only bet on GPG for teams that score a lot

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
anaheim = pd.read_csv('../Data/2017_2018/2017_2018_Anaheim.csv'); anaheim.index += 1
arizona = pd.read_csv('../Data/2017_2018/2017_2018_Arizona.csv'); arizona.index += 1
boston = pd.read_csv('../Data/2017_2018/2017_2018_Boston.csv'); boston.index += 1
tampa = pd.read_csv('../Data/2017_2018/2017_2018_Tampa.csv'); tampa.index += 1
washington = pd.read_csv('../Data/2017_2018/2017_2018_Washington.csv'); washington.index += 1
winnipeg = pd.read_csv('../Data/2017_2018/2017_2018_Winnipeg.csv'); winnipeg.index += 1

In [4]:
team_list = [anaheim, arizona, boston, tampa, washington, winnipeg]

In [5]:
all_teams = pd.read_csv('../Data/2017_2018/2017_2018_All_teams.csv'); all_teams.index += 1
all_teams = all_teams[ ['Tm', 'CF', 'CA', 'CF%', 'FF', 'FA', 'FF%'] ]
all_teams.rename(columns={'Tm':'Opp'}, inplace=True)

total_results = np.array(['Tm', 'CA', 'FA'])
initial_len = np.size(total_results)

for team in team_list:
    team = team[ ['Tm', 'Date', 'Opp', 'Result', 'Length', 'G'] ].copy()
    print(team['Tm'].iloc[0])
    total_results = np.append(total_results, team['Tm'].iloc[0])
    # if game went to OT/SO and Tampa won, then Score -= 1
    team.loc[(team.Result == 'W') & pd.notnull(team.Length) , 'G'] = team.G - 1 
    # merge team with all_teams
    merged_df = pd.merge(team, all_teams, on='Opp'); merged_df.index +=1

    y_column = merged_df.loc[:,'G']
    y_vals = y_column.values

    for stat in total_results[1:initial_len]:
        x_column = merged_df.loc[:,stat]
        x_vals = x_column.values
        r = np.round(np.corrcoef(x_vals, y_vals)[0][1], 5)
        total_results = np.append(total_results, r)
        print('\t', stat, 'r =', r)

ANA
	 CA r = 0.05686
	 FA r = 0.0886
ARI
	 CA r = 0.01449
	 FA r = -0.00285
BOS
	 CA r = 0.01838
	 FA r = 0.02449
TBL
	 CA r = 0.12096
	 FA r = 0.1078
WSH
	 CA r = 0.07999
	 FA r = 0.00795
WPG
	 CA r = 0.08559
	 FA r = 0.06034


In [6]:
total_results = total_results.reshape(len(team_list)+1, initial_len)
pd.DataFrame(total_results)

,0,1,2
0,Tm,CA,FA
1,ANA,0.05686,0.0886
2,ARI,0.01449,-0.00285
3,BOS,0.01838,0.02449
4,TBL,0.12096,0.1078
5,WSH,0.07999,0.00795
6,WPG,0.08559,0.06034
